# Chapter 7: DETECT (DE) – Continuous Monitoring & Discovery

In Chapter 6, we constructed comprehensive protective controls—fortifying identities with MFA, encrypting data at rest and in transit, hardening applications, and institutionalizing secure development practices. These defenses are robust, but the reality of cybersecurity is that prevention eventually fails. Zero-day vulnerabilities emerge, credentials are compromised through phishing, and sophisticated adversaries find ways to bypass even the most stringent controls. The assumption of breach is fundamental to modern security architecture.

The **DETECT** function of the NIST CSF 2.0 establishes the capabilities to identify cybersecurity events in a timely manner. Detection is the bridge between protection and response; it is the alarm system that alerts us when an intruder has bypassed our locks, the anomaly detection that spots the insider threat, and the forensic logging that reconstructs the timeline of a breach. Without detection, breaches linger undiscovered for months (the average dwell time in 2025 was 194 days), allowing adversaries to exfiltrate data, establish persistence, and move laterally with impunity.

For developers, detection is often misunderstood as purely an operations or SOC (Security Operations Center) function. This chapter dispels that myth. We will explore how to instrument applications to produce security-relevant telemetry, how to structure logs for machine analysis, how to integrate runtime protection directly into applications, and how anomaly detection algorithms identify compromised accounts and insider threats. By the end, you will understand that security is not just about building walls, but about installing sensors, cameras, and tripwires throughout your digital estate.

---

## 7.1 Security Logging: What, When, and How to Log Effectively (OWASP A09)

Logging and monitoring failures (OWASP A09: 2021) are a critical vulnerability. Without logs, we cannot detect breaches, debug security incidents, or meet compliance requirements. However, logging everything creates noise; logging too little creates blind spots.

### 7.1.1 What to Log: The Security Event Taxonomy
Not all events are security-relevant. Focus on the **5 Ws** of security logging:

**Authentication Events:**
*   Successful and failed login attempts
*   Password changes and resets
*   MFA challenges and outcomes
*   Session creation, validation, and destruction
*   Privilege escalation attempts

**Authorization Events:**
*   Access to sensitive resources (read/write/delete)
*   Permission changes (RBAC modifications)
*   Access denials (indicates potential reconnaissance)

**Data Events:**
*   Creation, modification, and deletion of sensitive data
*   Bulk data access (potential exfiltration)
*   Data classification changes

**System Events:**
*   Configuration changes (firewall rules, security group modifications)
*   Software installations and updates
*   Service starts and stops
*   Administrative actions

**Network Events:**
*   Inbound connections from unusual geographies
*   Outbound connections to command-and-control (C2) servers
*   Port scanning activities

### 7.1.2 Log Format: Structured Logging
Unstructured logs (plain text) are difficult to parse and correlate. **Structured logging** (JSON) enables automated analysis and SIEM ingestion.

**Implementation: Python Structured Security Logging**
```python
import logging
import json
from datetime import datetime
from pythonjsonlogger import jsonlogger

class SecurityLogger:
    def __init__(self, service_name):
        self.logger = logging.getLogger("security_audit")
        self.logger.setLevel(logging.INFO)
        
        # JSON formatter for SIEM ingestion
        logHandler = logging.StreamHandler()
        formatter = jsonlogger.JsonFormatter(
            '%(timestamp)s %(level)s %(service)s %(event_type)s %(user_id)s '
            '%(resource)s %(action)s %(result)s %(ip_address)s %(session_id)s '
            '%(message)s',
            rename_fields={'levelname': 'severity'}
        )
        logHandler.setFormatter(formatter)
        self.logger.addHandler(logHandler)
        
        self.service_name = service_name
    
    def log_auth_event(self, event_type, user_id, result, ip_address, 
                      session_id=None, additional_context=None):
        """
        Standardized authentication logging
        """
        log_data = {
            "timestamp": datetime.utcnow().isoformat() + "Z",
            "service": self.service_name,
            "event_type": event_type,  # LOGIN_SUCCESS, LOGIN_FAILURE, LOGOUT, etc.
            "user_id": self._hash_user_id(user_id),  # Privacy protection
            "resource": "authentication_service",
            "action": event_type,
            "result": result,  # SUCCESS, FAILURE, DENIED
            "ip_address": ip_address,
            "session_id": session_id or "N/A",
            "user_agent": additional_context.get('user_agent') if additional_context else None,
            "mfa_used": additional_context.get('mfa_used', False) if additional_context else False
        }
        
        # Include failure reason for failed attempts (but not sensitive data)
        if result == "FAILURE" and additional_context:
            log_data["failure_reason"] = additional_context.get('reason', 'Unknown')
        
        self.logger.info("Authentication event", extra=log_data)
        
        # Critical: Log to tamper-resistant store (WORM storage) for compliance
        self._write_to_immutable_store(log_data)
    
    def log_data_access(self, user_id, resource, action, classification, 
                       records_accessed, ip_address):
        """
        Log access to sensitive data for DLP and privacy compliance
        """
        log_data = {
            "timestamp": datetime.utcnow().isoformat() + "Z",
            "service": self.service_name,
            "event_type": "DATA_ACCESS",
            "user_id": self._hash_user_id(user_id),
            "resource": resource,
            "action": action,  # READ, WRITE, DELETE, EXPORT
            "data_classification": classification,  # RESTRICTED, CONFIDENTIAL, etc.
            "records_count": records_accessed,
            "ip_address": ip_address,
            "anomaly_score": self._calculate_anomaly_score(user_id, resource)
        }
        
        # High-volume access alerts
        if records_accessed > 1000:
            log_data["alert"] = "BULK_ACCESS_DETECTED"
        
        self.logger.info("Data access event", extra=log_data)
    
    def _hash_user_id(self, user_id):
        """One-way hash of user ID for privacy in logs (pseudonymization)"""
        import hashlib
        return hashlib.sha256(user_id.encode()).hexdigest()[:16]
    
    def _write_to_immutable_store(self, log_data):
        """Write to Write-Once-Read-Many (WORM) storage for forensic integrity"""
        # Implementation would write to AWS S3 Glacier WORM or similar
        pass

# Usage
sec_logger = SecurityLogger("customer_portal")
sec_logger.log_auth_event(
    event_type="LOGIN_ATTEMPT",
    user_id="user@example.com",
    result="FAILURE",
    ip_address="203.0.113.45",
    additional_context={
        "reason": "INVALID_PASSWORD",
        "mfa_used": False,
        "user_agent": "Mozilla/5.0..."
    }
)
```

### 7.1.3 Secure Logging: Protecting the Logs
Logs are high-value targets; attackers delete logs to cover tracks.

**Protection Measures:**
*   **Integrity:** Cryptographic signing of log entries (hash chains).
*   **Confidentiality:** Encrypt logs containing sensitive data (PII).
*   **Availability:** Forward to centralized, append-only storage immediately.
*   **Access Control:** Principle of least privilege; developers can read, not delete.

**Implementation: Log Integrity with Hash Chains**
```python
import hashlib
import hmac

class TamperEvidentLogger:
    def __init__(self, secret_key):
        self.previous_hash = "0" * 64  # Genesis hash
        self.secret_key = secret_key
    
    def create_log_entry(self, message):
        """
        Creates a tamper-evident log entry using hash chaining
        Similar to blockchain integrity mechanism
        """
        timestamp = datetime.utcnow().isoformat()
        
        # Concatenate previous hash with current message
        entry_data = f"{self.previous_hash}|{timestamp}|{message}"
        
        # HMAC for integrity (not just SHA256 to prevent length extension attacks)
        current_hash = hmac.new(
            self.secret_key.encode(),
            entry_data.encode(),
            hashlib.sha256
        ).hexdigest()
        
        entry = {
            "timestamp": timestamp,
            "message": message,
            "previous_hash": self.previous_hash,
            "entry_hash": current_hash
        }
        
        self.previous_hash = current_hash
        return entry
    
    def verify_chain(self, log_entries):
        """
        Verify integrity of log chain
        Returns: (is_valid, first_invalid_index)
        """
        expected_hash = "0" * 64
        
        for i, entry in enumerate(log_entries):
            reconstructed = f"{entry['previous_hash']}|{entry['timestamp']}|{entry['message']}"
            calculated_hash = hmac.new(
                self.secret_key.encode(),
                reconstructed.encode(),
                hashlib.sha256
            ).hexdigest()
            
            if calculated_hash != entry['entry_hash']:
                return False, i
            
            expected_hash = entry['entry_hash']
        
        return True, -1
```

### 7.1.4 Logging in Cloud-Native Environments
Cloud services generate their own logs that must be aggregated:

*   **AWS:** CloudTrail (API calls), VPC Flow Logs (network), S3 Access Logs.
*   **Azure:** Activity Logs, Diagnostic Logs, NSG Flow Logs.
*   **GCP:** Cloud Audit Logs, VPC Flow Logs.

**Centralized Logging Architecture:**
```
Application Logs → Fluentd/Fluent Bit → Kafka → Logstash → Elasticsearch/ClickHouse → Kibana/Grafana
CloudWatch Logs → S3 → Lambda Processing → SIEM
```

---

## 7.2 SIEM Fundamentals: Aggregation, Correlation, and Alerting

A **Security Information and Event Management (SIEM)** system is the central nervous system of detection, aggregating logs from across the enterprise to identify patterns indicative of attacks.

### 7.2.1 SIEM Architecture
1.  **Collection:** Log forwarders (agents) on endpoints, network taps, cloud APIs.
2.  **Normalization:** Parsing diverse formats into a common schema (Common Event Format - CEF, or JSON).
3.  **Enrichment:** Adding context (GeoIP, threat intelligence, asset criticality).
4.  **Correlation:** Matching events across time and systems (e.g., "5 failed logins followed by a successful login from a new location").
5.  **Storage:** Hot storage (recent, fast queries) and cold storage (archival, compliance).
6.  **Analysis:** Dashboards, alerting, and investigation workflows.

### 7.2.2 Correlation Rules and Use Cases
**Example Correlation Rules:**

**Brute Force Detection:**
```
Event: Authentication Failure
Threshold: 5 failures within 5 minutes from same IP
Action: Alert SOC, block IP at firewall
```

**Lateral Movement Detection:**
```
Event Sequence:
1. Login to Workstation A (User: Alice)
2. Within 1 hour: SMB connection from Workstation A to Server B (User: Alice)
3. Server B never accessed by Alice before
Action: Alert potential lateral movement
```

**Impossible Travel:**
```
Event: Login from New York at 09:00 EST
Event: Login from Tokyo at 09:30 EST (30 minutes later)
Physics: Impossible to travel 6,700 miles in 30 minutes
Action: Alert potential credential theft or VPN bypass
```

**Implementation: Sigma Rules (Generic SIEM Signatures)**
Sigma is a generic signature format for SIEM systems (Splunk, Elastic, etc.).

```yaml
# Detects suspicious PowerShell execution (often used in malware)
title: Suspicious PowerShell Execution
status: stable
description: Detects suspicious PowerShell execution with encoded commands or bypass flags
logsource:
    category: process_creation
    product: windows
detection:
    selection:
        CommandLine|contains:
            - '-enc '
            - '-encodedcommand '
            - 'bypass'
            - '-nop'
            - 'hidden'
        Image|endswith: '\powershell.exe'
    condition: selection
falsepositives:
    - Administrative scripts
level: high
tags:
    - attack.execution
    - attack.t1059.001  # MITRE ATT&CK: PowerShell
```

### 7.2.3 Alert Fatigue and Tuning
The greatest enemy of detection is **alert fatigue**—too many false positives causing analysts to ignore real threats.

**Best Practices:**
*   **Baseline:** Establish normal behavior before alerting on anomalies.
*   **Severity:** Only "Critical" and "High" alerts wake people up at night.
*   **Automation:** Low-risk alerts trigger automated responses (blocking IPs), not human investigation.
*   **Threat-Centric:** Tune rules to your threat model (financial sector cares about wire fraud, healthcare about PHI access).

---

## 7.3 Application Security Monitoring (ASM) and RASP

While network and host monitoring are essential, applications require specialized monitoring that understands code-level behavior.

### 7.3.1 Application Security Monitoring (ASM)
ASM focuses on the application layer:
*   **Business Logic Abuse:** Detecting abuse of legitimate features (coupon fraud, scraping, fake account creation).
*   **API Abuse:** Unusual patterns in API calls (sequencing attacks, parameter tampering).
*   **User Behavior:** Session hijacking indicators (sudden change in user agent, location, or behavior patterns).

### 7.3.2 Runtime Application Self-Protection (RASP)
RASP embeds security controls within the application runtime, intercepting calls to the OS, databases, and file systems to detect and block attacks in real-time.

**How RASP Works:**
1.  Instruments the runtime (JVM, .NET CLR, Python interpreter, or Node.js).
2.  Intercepts all I/O operations (database queries, file reads, network connections).
3.  Applies security policies (e.g., "this query looks like SQL injection").
4.  Blocks malicious requests and logs the attempt.

**Implementation: Conceptual RASP Hook (Python Monkey-Patching Example)**
```python
import functools
import re

class RASPMonitor:
    """
    Conceptual RASP implementation using function wrapping
    Real RASP uses runtime instrumentation (e.g., Java agents, Python sys.settrace)
    """
    
    SQLI_PATTERNS = [
        r"(\%27)|(\')|(\-\-)|(\%23)|(#)",  # Basic SQL meta-characters
        r"((\%3D)|(=))[^\n]*((\%27)|(\')|(\-\-)|(\%3B)|(;))",  # Equals followed by quotes
        r"\w*((\%27)|(\'))((\%6F)|o|(\%4F))((\%72)|r|(\%52))",  # 'or' variations
    ]
    
    def __init__(self):
        self.blocked_attempts = []
    
    def protect_db_cursor(self, cursor_class):
        """
        Wrap database cursor execute method to detect SQL injection
        """
        original_execute = cursor_class.execute
        
        @functools.wraps(original_execute)
        def secure_execute(self_cursor, query, parameters=None):
            # Check for SQL injection in raw query
            if isinstance(query, str):
                for pattern in RASPMonitor.SQLI_PATTERNS:
                    if re.search(pattern, query, re.IGNORECASE):
                        self._alert_and_block("SQL_INJECTION_DETECTED", query)
                        raise SecurityException("Malicious query blocked by RASP")
            
            # Check for second-order injection (parameters)
            if parameters:
                param_str = str(parameters)
                for pattern in RASPMonitor.SQLI_PATTERNS:
                    if re.search(pattern, param_str, re.IGNORECASE):
                        self._alert_and_block("SQLI_IN_PARAMETER", param_str)
                        raise SecurityException("Malicious parameters blocked")
            
            return original_execute(self_cursor, query, parameters)
        
        cursor_class.execute = secure_execute
        return cursor_class
    
    def _alert_and_block(self, attack_type, payload):
        """
        Log and alert on blocked attack
        """
        alert = {
            "timestamp": datetime.utcnow().isoformat(),
            "severity": "CRITICAL",
            "attack_type": attack_type,
            "payload_hash": hashlib.sha256(payload.encode()).hexdigest(),
            "action": "BLOCKED",
            "stack_trace": traceback.format_exc()
        }
        
        # Send to SIEM immediately
        siem_client.send_alert(alert)
        
        # Increment metrics for monitoring
        metrics.increment(f"rasp.blocked.{attack_type}")

# Usage: Initialize RASP protection
# RASPMonitor().protect_db_cursor(psycopg2.extensions.cursor)
```

**RASP vs WAF:**
*   **WAF (Web Application Firewall):** Network perimeter, inspects HTTP traffic. Can be bypassed with encoding.
*   **RASP:** Inside the application, sees decoded, processed data. Cannot be bypassed by network tricks, but consumes application resources.

### 7.3.3 Interactive Application Security Testing (IAST)
IAST combines SAST (static) and DAST (dynamic) by instrumenting the application during testing to detect vulnerabilities in real-time with full code context (line numbers, data flow).

---

## 7.4 Threat Intelligence Feeds: Integrating External Context

Threat intelligence provides context about external threats—who is attacking, what tools they use, and what infrastructure they command.

### 7.4.1 Types of Threat Intelligence
**Tactical:** IOCs (Indicators of Compromise) - IPs, domains, file hashes.
**Operational:** TTPs (Tactics, Techniques, and Procedures) - how attackers operate.
**Strategic:** High-level trends (ransomware group activities, nation-state APT campaigns).

### 7.4.2 IOCs and Enrichment
**IOC Types:**
*   **Network:** IP addresses, C2 domains, URLs.
*   **Host:** File hashes (MD5, SHA256), registry keys, mutex names.
*   **Email:** Sender addresses, subject lines, attachment hashes.

**Implementation: Threat Intelligence Integration**
```python
class ThreatIntelEnricher:
    def __init__(self):
        # Connect to threat intel feeds (MISP, VirusTotal, AlienVault OTX)
        self.misp_client = MISPClient(api_key=os.environ['MISP_KEY'])
        self.vt_client = VirusTotalClient(api_key=os.environ['VT_KEY'])
    
    def check_ip_reputation(self, ip_address):
        """
        Check if IP is known malicious
        """
        reputation = {
            "ip": ip_address,
            "is_malicious": False,
            "sources": [],
            "confidence": 0
        }
        
        # Check MISP (Malware Information Sharing Platform)
        misp_result = self.misp_client.search_ip(ip_address)
        if misp_result:
            reputation["is_malicious"] = True
            reputation["sources"].append("MISP")
            reputation["confidence"] = max(reputation["confidence"], misp_result['confidence'])
        
        # Check VirusTotal
        vt_result = self.vt_client.get_ip_report(ip_address)
        if vt_result['malicious_votes'] > 5:
            reputation["is_malicious"] = True
            reputation["sources"].append("VirusTotal")
            reputation["confidence"] = max(
                reputation["confidence"], 
                vt_result['malicious_votes'] / vt_result['total_votes']
            )
        
        return reputation
    
    def enrich_log_entry(self, log_entry):
        """
        Enrich log entry with threat intelligence
        """
        if 'ip_address' in log_entry:
            ip_rep = self.check_ip_reputation(log_entry['ip_address'])
            log_entry['threat_intel'] = ip_rep
            
            # Auto-escalate if malicious
            if ip_rep['is_malicious'] and log_entry['result'] == 'SUCCESS':
                log_entry['alert_level'] = 'CRITICAL'
                log_entry['alert_reason'] = 'Login from known malicious IP'
        
        return log_entry
```

### 7.4.3 MITRE ATT&CK Mapping
Map detection rules to **MITRE ATT&CK** techniques to identify coverage gaps.

**Example Mapping:**
*   **Detection:** "PowerShell with encoded command"
*   **ATT&CK:** T1059.001 (Command and Scripting Interpreter: PowerShell)
*   **Tactic:** Execution

**Benefits:**
*   Identify which techniques you can detect vs. which you cannot.
*   Report to management: "We detect 85% of Initial Access techniques but only 40% of Persistence techniques."

---

## 7.5 Anomaly Detection: Network, User, and Entity Behavior Analytics (UEBA)

Signature-based detection (known bad patterns) cannot catch novel attacks. **Anomaly detection** (behavioral analysis) establishes baselines of normal activity and flags deviations.

### 7.5.1 UEBA Concepts
**UEBA** analyzes:
*   **Users:** What resources do they access? When? From where?
*   **Entities:** Servers, databases, applications—what do they connect to?
*   **Peers:** Is this user behaving like their peer group?

**Anomaly Types:**
*   **Time-based:** Activity at unusual hours (3 AM login for 9-5 worker).
*   **Volume-based:** Downloading 100x normal data volume.
*   **Pattern-based:** Accessing resources never touched before (new department files).
*   **Graph-based:** Unusual connection patterns (hub-and-spoke lateral movement).

### 7.5.2 Statistical Anomaly Detection
Using statistical methods to identify outliers.

**Implementation: Simple Statistical Anomaly Detector**
```python
import numpy as np
from scipy import stats
from collections import defaultdict

class BehavioralAnalyzer:
    def __init__(self):
        # Store baselines: user -> metric -> historical values
        self.baselines = defaultdict(lambda: defaultdict(list))
        self.window_size = 30  # 30 days of history
    
    def update_baseline(self, user_id, metric, value):
        """
        Update user's behavioral baseline
        """
        history = self.baselines[user_id][metric]
        history.append(value)
        
        # Keep only recent history (sliding window)
        if len(history) > self.window_size:
            history.pop(0)
    
    def is_anomalous(self, user_id, metric, current_value, threshold=3.0):
        """
        Detect anomaly using Z-score (standard deviations from mean)
        threshold=3.0 means 99.7% confidence interval
        """
        history = self.baselines[user_id][metric]
        
        if len(history) < 5:  # Insufficient data
            return False, 0
        
        mean = np.mean(history)
        std = np.std(history)
        
        if std == 0:  # No variation in history
            return current_value != mean, 0
        
        z_score = (current_value - mean) / std
        
        # Also check for sudden drop (possible account abandonment)
        is_anomaly = abs(z_score) > threshold
        
        return is_anomaly, z_score
    
    def detect_data_exfiltration(self, user_id, bytes_transferred, files_accessed):
        """
        Specific detection for potential data exfiltration
        """
        alerts = []
        
        # Check volume anomaly
        volume_anomaly, volume_z = self.is_anomalous(
            user_id, 'daily_bytes', bytes_transferred, threshold=3
        )
        
        if volume_anomaly and bytes_transferred > 100000000:  # >100MB
            alerts.append({
                'type': 'VOLUME_ANOMALY',
                'severity': 'HIGH',
                'z_score': volume_z,
                'details': f"User transferred {bytes_transferred} bytes (normal: {np.mean(self.baselines[user_id]['daily_bytes'])})"
            })
        
        # Check file access diversity (accessing many different files)
        file_anomaly, file_z = self.is_anomalous(
            user_id, 'files_accessed_count', files_accessed, threshold=2.5
        )
        
        if file_anomaly:
            alerts.append({
                'type': 'SCOPE_ANOMALY',
                'severity': 'MEDIUM',
                'details': f"Accessed {files_accessed} unique files (unusual breadth)"
            })
        
        # Check for access outside business hours (time anomaly)
        hour = datetime.now().hour
        if hour < 7 or hour > 19:  # Before 7 AM or after 7 PM
            if not self._is_oncall(user_id):  # Check if scheduled
                alerts.append({
                    'type': 'TIME_ANOMALY',
                    'severity': 'MEDIUM',
                    'details': f"Activity at {hour}:00 (outside normal hours)"
                })
        
        return alerts
    
    def _is_oncall(self, user_id):
        # Integration with PagerDuty/Opsgenie API
        return False  # Simplified

# Usage
# analyzer = BehavioralAnalyzer()
# analyzer.update_baseline('user123', 'daily_bytes', 5000000)  # 5MB normal
# 
# # Later, check current activity
# is_anomaly, z = analyzer.is_anomalous('user123', 'daily_bytes', 500000000)  # 500MB
# if is_anomaly:
#     alert_security_team()
```

### 7.5.3 Machine Learning for Anomaly Detection
Advanced UEBA uses ML:
*   **Clustering:** Grouping similar users; flagging outliers.
*   **Graph Analysis:** Detecting unusual connection paths (lateral movement).
*   **Deep Learning:** Autoencoders that learn complex normal patterns and flag reconstruction errors.

**Challenges:**
*   **False Positives:** Novel but legitimate behavior (new project) triggers alerts.
*   **Adversarial Adaptation:** Attackers learn to mimic normal behavior ("low and slow").
*   **Explainability:** ML models must explain why something is anomalous for SOC analysts to act.

---

### Chapter Summary

In this chapter, we implemented the **DETECT** function, establishing comprehensive visibility into our security posture. We implemented **structured security logging** with tamper-evident hash chains, ensuring forensic integrity while protecting user privacy through pseudonymization. We explored **SIEM architecture**, understanding how correlation rules transform isolated events into actionable intelligence, while managing the critical challenge of alert fatigue through risk-based tuning. We examined **Application Security Monitoring** and **RASP**, embedding detection capabilities directly within applications to block attacks that bypass perimeter defenses. We integrated **Threat Intelligence** to contextualize internal events with external adversary knowledge, mapping our detections to the **MITRE ATT&CK** framework to identify coverage gaps. Finally, we implemented **UEBA** statistical anomaly detection to identify compromised accounts and insider threats through behavioral baselines.

Detection alerts us to the presence of adversaries, but detection alone cannot contain damage. When the alarm sounds—when the RASP blocks an SQL injection attempt, when the SIEM correlates a lateral movement pattern, when the UEBA flags an impossible travel scenario—we must act. We must contain the threat, eradicate the presence, and restore services. We transition from observation to intervention, from the SOC analyst's dashboard to the incident responder's playbook.

**Next Up: Chapter 8: RESPOND (RS) – Incident Management**

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='6. protect_pr_safeguards_implementation.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='8. respond_rs_incident_management.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
